# Step5: Post-Processing function step 2

## These series of notebooks rely on the 'load-traffic' branch of the GOSTnets repo and has not yet been merged into master. The link to the code is here: https://github.com/worldbank/GOSTnets/tree/load_traffic

In [50]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTnets")
import GOSTnets as gn

from shapely.geometry import LineString, Point

In [51]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
from GOSTnets.load_traffic2 import *

In [54]:
# import shapefile as GPD
study_area = gpd.read_file("./output_edges_merged/cities_weighted_sec_saved_edges_all_merged_max_speed.shp")

In [55]:
study_area

,o,d,w,sec_saved,imp_cost,mean_speed,length,infra_type,weighted_s,max_speed,min_speed,mode,osm_id,speed,time,geometry
0,308865184,4154326634,4,0.0,0.0,NaN,0.123787,trunk,0.0,NaN,NaN,drive,779522233.0,60.0,7.427222,"LINESTRING (79.83202 8.02366, 79.83152 8.02466)"
1,308865185,3693219150,4,0.0,0.0,NaN,0.013228,trunk,0.0,NaN,NaN,drive,779522233.0,60.0,0.793704,"LINESTRING (79.83662 8.01453, 79.83656 8.01463)"
2,308865186,2092300926,4,0.0,0.0,NaN,0.106519,trunk,0.0,NaN,NaN,drive,779522233.0,60.0,6.391161,"LINESTRING (79.83761 8.01102, 79.83737 8.01195)"
3,308865187,2092300773,4,0.0,0.0,32.0,0.021977,trunk,0.0,32.0,32.0,drive,779522233.0,32.0,2.472436,"LINESTRING (79.83836 8.00608, 79.83834 8.00628)"
4,308865188,3693217644,3,0.0,0.0,NaN,0.051264,trunk,0.0,NaN,NaN,drive,779522233.0,60.0,3.075858,"LINESTRING (79.83823 8.00407, 79.83830 8.00453)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77942,7291030420,7291030424,1,0.0,0.0,46.0,0.023952,primary,0.0,46.0,46.0,drive,50286645.0,46.0,1.874513,"LINESTRING (79.89913 7.08844, 79.89897 7.08829)"
77943,7291030421,7291030420,1,0.0,0.0,46.0,0.060993,primary,0.0,46.0,46.0,drive,50286645.0,46.0,4.773354,"LINESTRING (79.89949 7.08886, 79.89913 7.08844)"
77944,7291030422,7291030416,1,0.0,0.0,55.0,0.029824,primary,0.0,55.0,55.0,drive,50286645.0,55.0,1.952104,"LINESTRING (79.90008 7.08948, 79.89989 7.08928)"
77945,7291030423,7291030400,1,0.0,0.0,NaN,0.026295,primary,0.0,NaN,NaN,drive,50286645.0,60.0,1.577705,"LINESTRING (79.90041 7.08982, 79.90025 7.08965)"


In [34]:
sorted_study_area = study_area.sort_values(by=['weighted_s'], ascending=False)

In [35]:
sorted_study_area[:15]

,o,d,w,sec_saved,imp_cost,mean_speed,length,infra_type,weighted_s,max_speed,min_speed,mode,osm_id,speed,time,geometry
21886,2404808254,2085208123,1,844.404586,2700.497511,3.327381,0.865125,secondary,844.404586,17.0,3.0,drive,696878662.0,3.327381,936.006015,"MULTILINESTRING ((80.71650 6.74198, 80.71664 6..."
22651,3157384370,3182624304,1,839.941994,2198.884977,11.565476,4.089520,secondary,839.941994,30.0,11.0,drive,310319285.0,11.565476,1272.950032,"MULTILINESTRING ((81.01818 6.74353, 81.01822 6..."
21824,2404808172,6544528194,1,483.584620,15756.683720,11.952058,3.099901,tertiary,483.584620,12.0,12.0,drive,232117179.0,12.000000,929.970422,"MULTILINESTRING ((80.73259 6.74094, 80.73279 6..."
50880,1157613973,3999447280,1,439.760211,3226.938775,14.000000,2.907304,secondary,439.760211,14.0,14.0,drive,397124613.0,14.000000,747.592359,"MULTILINESTRING ((80.68311 7.41676, 80.68323 7..."
21757,2401644262,4136269381,5,80.874419,1526.354046,22.105049,1.422010,secondary,404.372097,26.0,22.0,drive,353299188.0,22.119048,231.440222,"MULTILINESTRING ((80.93071 6.77846, 80.93070 6..."
75451,1775834997,1775835028,1,394.795557,8498.884201,1.387669,0.269575,tertiary,394.795557,17.0,2.0,drive,166100090.0,2.238095,433.614367,"MULTILINESTRING ((81.24036 8.57467, 81.23999 8..."
69849,5176027961,4456327916,2,179.458499,1821.417966,18.852393,2.146855,secondary,358.916998,19.0,19.0,drive,310456241.0,19.000000,406.772598,"MULTILINESTRING ((80.20028 7.07048, 80.20022 7..."
75739,1884656111,3350311401,3,144.176462,11492.176915,20.798404,5.257986,tertiary,332.508536,23.0,20.0,drive,178088801.0,21.000992,901.326429,"MULTILINESTRING ((81.84543 7.29459, 81.84597 7..."
74324,1172970454,1240466608,16,18.257237,46014.034292,20.000000,0.246328,secondary,292.115799,20.0,20.0,drive,101600843.0,20.000000,44.339005,"LINESTRING (81.22839 8.58093, 81.23036 8.57988)"
79628,4366114665,1242791243,21,13.038323,11275.735255,10.000000,0.060363,tertiary,273.804792,10.0,10.0,drive,188699615.0,10.000000,21.730539,"LINESTRING (81.23471 8.57738, 81.23462 8.57684)"


In [57]:
sorted_study_area['w_sec_per_km']=sorted_study_area['weighted_s'] / sorted_study_area['length']

## calculate w_sec_per_km

In [74]:
#sorted_study_area

In [82]:
bucket1 = sorted_study_area.loc[sorted_study_area['length'] < .015]

In [83]:
bucket1.to_file(driver = 'ESRI Shapefile', filename = "output_edges_merged/cities_weighted_sec_saved_edges_all_merged_traffic_max_speed_bucket1.shp")

In [84]:
bucket2 = sorted_study_area.loc[(sorted_study_area['length'] > .015) & (sorted_study_area['length'] < .035)]

In [85]:
bucket2.to_file(driver = 'ESRI Shapefile', filename = "output_edges_merged/cities_weighted_sec_saved_edges_all_merged_traffic_max_speed_bucket2.shp")

In [86]:
bucket3 = sorted_study_area.loc[(sorted_study_area['length'] > .035)]

In [87]:
bucket3.to_file(driver = 'ESRI Shapefile', filename = "output_edges_merged/cities_weighted_sec_saved_edges_all_merged_traffic_max_speed_bucket3.shp")

In [45]:
total_cost = 0
total_count = 0

In [46]:
def sort_weighted_s(sec_saved, imp_cost):
    global total_cost
    global total_count
    print(sec_saved)
    if total_cost < 20000000 and sec_saved > 0:
        print(f"total_cost before:{total_cost}")
        print(f"imp_cost:{imp_cost}")
        total_cost = total_cost + imp_cost
        total_count += 1
        print(f"sec_saved:{sec_saved}")
        print(f"total_cost:{total_cost}")
        print(f"total_count:{total_count}")


In [47]:
#sorted_study_area['weighted_s'] = 
sorted_study_area.apply(lambda x: sort_weighted_s(x['sec_saved'],x['imp_cost']),axis=1)

844.4045862180109
total_cost before:0
imp_cost:2700.497510811517
sec_saved:844.4045862180109
total_cost:2700.497510811517
total_count:1
839.9419944670673
total_cost before:2700.497510811517
imp_cost:2198.884976559355
sec_saved:839.9419944670673
total_cost:4899.382487370872
total_count:2
483.58461958772335
total_cost before:4899.382487370872
imp_cost:15756.683719832608
sec_saved:483.58461958772335
total_cost:20656.06620720348
total_count:3
439.76021116179044
total_cost before:20656.06620720348
imp_cost:3226.938774519543
sec_saved:439.76021116179044
total_cost:23883.00498172302
total_count:4
80.87441931702607
total_cost before:23883.00498172302
imp_cost:1526.354045848347
sec_saved:80.87441931702607
total_cost:25409.35902757137
total_count:5
394.7955567168531
total_cost before:25409.35902757137
imp_cost:8498.884201246507
sec_saved:394.7955567168531
total_cost:33908.243228817875
total_count:6
179.45849915069167
total_cost before:33908.243228817875
imp_cost:1821.4179662843178
sec_saved:179.

5.13260321247948
total_cost before:2747275.0994166713
imp_cost:7846.812725997789
sec_saved:5.13260321247948
total_cost:2755121.9121426693
total_count:348
5.110641161424983
total_cost before:2755121.9121426693
imp_cost:9759.581754589053
sec_saved:5.110641161424983
total_cost:2764881.4938972583
total_count:349
5.080520857368012
total_cost before:2764881.4938972583
imp_cost:1351.1989461329808
sec_saved:5.080520857368012
total_cost:2766232.692843391
total_count:350
5.076199223567742
total_cost before:2766232.692843391
imp_cost:6006.607283769935
sec_saved:5.076199223567742
total_cost:2772239.300127161
total_count:351
0.84226315774441
total_cost before:2772239.300127161
imp_cost:4234.198849727462
sec_saved:0.84226315774441
total_cost:2776473.4989768886
total_count:352
1.669609168873627
total_cost before:2776473.4989768886
imp_cost:8396.221109735128
sec_saved:1.669609168873627
total_cost:2784869.720086624
total_count:353
2.504261753882238
total_cost before:2784869.720086624
imp_cost:12325.337

2.364724344013749
total_cost before:4083035.9766503572
imp_cost:16940.136751909835
sec_saved:2.364724344013749
total_cost:4099976.113402267
total_count:532
1.17982017958278
total_cost before:4099976.113402267
imp_cost:4894.5598282551455
sec_saved:1.17982017958278
total_cost:4104870.6732305223
total_count:533
0.155397346805168
total_cost before:4104870.6732305223
imp_cost:3551.6025979667315
sec_saved:0.155397346805168
total_cost:4108422.275828489
total_count:534
2.328146197708023
total_cost before:4108422.275828489
imp_cost:19167.7138733671
sec_saved:2.328146197708023
total_cost:4127589.9897018564
total_count:535
2.324723442111962
total_cost before:4127589.9897018564
imp_cost:4167.6386776289355
sec_saved:2.324723442111962
total_cost:4131757.628379485
total_count:536
2.31436903546184
total_cost before:4131757.628379485
imp_cost:6939.1278552487
sec_saved:2.31436903546184
total_cost:4138696.756234734
total_count:537
2.285118463170146
total_cost before:4138696.756234734
imp_cost:5536.640926

imp_cost:7313.555127945666
sec_saved:1.019154051507289
total_cost:5806958.642711378
total_count:790
0.144802534515027
total_cost before:5806958.642711378
imp_cost:8430.307024441938
sec_saved:0.144802534515027
total_cost:5815388.949735819
total_count:791
1.01284323803721
total_cost before:5815388.949735819
imp_cost:1588.3382650424508
sec_saved:1.01284323803721
total_cost:5816977.288000862
total_count:792
1.003315508364765
total_cost before:5816977.288000862
imp_cost:2386.2705751634808
sec_saved:1.003315508364765
total_cost:5819363.558576026
total_count:793
0.500662321015928
total_cost before:5819363.558576026
imp_cost:1114.1970287745062
sec_saved:0.500662321015928
total_cost:5820477.755604801
total_count:794
1.001235718109898
total_cost before:5820477.755604801
imp_cost:3443.4044780478885
sec_saved:1.001235718109898
total_cost:5823921.160082849
total_count:795
0.247549613286755
total_cost before:5823921.160082849
imp_cost:1048.158069271226
sec_saved:0.247549613286755
total_cost:5824969.

total_cost:7354761.754829003
total_count:1070
0.318595808057458
total_cost before:7354761.754829003
imp_cost:2188.4642732222965
sec_saved:0.318595808057458
total_cost:7356950.219102225
total_count:1071
0.158513865069636
total_cost before:7356950.219102225
imp_cost:8429.379513973441
sec_saved:0.158513865069636
total_cost:7365379.598616199
total_count:1072
0.31462992195693
total_cost before:7365379.598616199
imp_cost:9826.15202643893
sec_saved:0.31462992195693
total_cost:7375205.750642638
total_count:1073
0.156914267819552
total_cost before:7375205.750642638
imp_cost:2076.2372870323784
sec_saved:0.156914267819552
total_cost:7377281.98792967
total_count:1074
0.313368388925956
total_cost before:7377281.98792967
imp_cost:4674.84703535236
sec_saved:0.313368388925956
total_cost:7381956.834965022
total_count:1075
0.313237332605181
total_cost before:7381956.834965022
imp_cost:1047.445684733471
sec_saved:0.313237332605181
total_cost:7383004.280649756
total_count:1076
0.308841951655611
total_cost

...

-2.580841490879612
-0.738662771057062
-0.739091117784911
-2.588043116734204
-2.594051909195334
-0.741586744045663
-5.196962323012896
-5.198853893353464
-5.198896963499682
-5.211083826807252
-2.60775901416067
-2.61129423016525
-5.224839532006154
-5.22658943324427
-1.046807230458517
-0.249586383205719
-5.242676205304725
-0.749799892078981
-5.250924453847786
-0.750139114333572
-0.750408794727124
-0.750695242469776
-5.256197623421678
-5.25925701707185
-5.263407666516086
-5.26747840398438
-2.634327373388758
-0.752957384719025
-1.758585418399398
-5.285272000934731
-0.756076845842971
-1.059379340510642
-5.298092586751006
-5.299163522871283
-0.757286323950163
-5.302151007322246
-5.304016308212372
-0.758563332252072
-0.758756169725677
-1.771169912222621
-5.314539475547429
-5.317105292122024
-0.409121667594741
-5.325898243276829
-2.665133156980398
-5.330461404283392
-0.762064338047598
-1.778838513027982
-0.762645036611768
-5.350393341488253
-5.356120400474682
-2.680656202708447
-5.36968196731124

-1.866756705488454
-1.334349330042532
-1.33602848589723
-4.678918297557322
-4.681647033656433
-1.338456048214353
-1.338511303101934
-1.339881467594068
-9.383606019465102
-3.132753821251264
-4.705032372068631
-9.423806843863542
-9.481250672890104
-1.355014225541644
-3.165267806403689
-9.50214378205256
-1.357696181465224
-0.733328968654427
-4.773039336531125
-3.186992279150303
-0.736019026327881
-4.784637492380326
-4.787034721638056
-0.456713497963476
-0.481036423315562
-0.459453012063792
-1.378899080212834
-1.380885378972244
-0.483810553278735
-9.682232788478146
-1.383841721601785
-1.385077411954048
-4.850539748305105
-9.704308597249408
-4.852458107046973
-1.386670204060972
-0.462561013163676
-1.388137626810999
-1.388622409845066
-4.863410280500467
-1.392121655353952
-4.875130631749818
-1.625582288266299
-1.395560590369284
-1.396146927495831
-9.77893753081409
-9.786143102376988
-1.398142611454769
-9.796590332828501
-9.799170012595564
-9.799702883899092
-3.270558384667741
-1.402228217680

-21.251456611241053
-3.045295263432938
-1.018318424164935
-1.069420215973462
-1.6458029027735
-3.065477757754925
-21.47986316136207
-1.433787438646876
-1.024276317639147
-3.075808056238262
-1.438780328071061
-10.804120427828078
-7.214231089555541
-7.218394521745527
-10.837787226761659
-7.232301993889861
-7.245899980497374
-1.677317739279093
-3.115194683264622
-1.456546193480354
-21.87848217829017
-1.686559102034737
-1.463146540109063
-1.0465037660694
-22.019131314092423
-1.050264931529054
-22.29442452218888
-11.159089174046935
-1.488837034676693
-1.122555403783701
-1.069215381503456
-11.241150949588043
-22.488514589800374
-7.49768140570046
-1.07137875980815
-3.218928744407298
-1.502847649658433
-1.074482663324113
-1.744953957940059
-1.080788486397025
-1.137167447650289
-1.083455051541464
-1.085702862771014
-1.522153146603175
-1.142007490158686
-11.438045128606015
-3.26815205491043
-1.148143628100476
-1.095160925879179
-1.096795248716484
-1.771874376513924
-1.097003984384345
-3.30633728

-2.805057164040235
-2.81083720229363
-2.813401418947854
-2.698210348178764
-2.835690495644634
-2.712028070241109
-28.571874790416018
-8.181028295441916
-2.72846533000592
-2.870224630292449
-2.871537113719515
-2.880212015922918
-2.745881520322034
-2.749822588102688
-2.889110243316384
-2.893766982464552
-2.900555992135791
-2.763248675425076
-2.773078163464471
-2.924228500140051
-2.924939390362693
-2.925684201486381
-2.804213646043666
-2.807910614035055
-2.811855405090717
-2.826717968320827
-2.853525887088958
-2.996749436962173
-19.986590028099833
-2.855919218180917
-3.001984235737554
-3.007723685821562
-3.009818461836897
-3.019988253989928
-3.027869203307663
-3.029459214275268
-3.03340195076954
-2.894409874717684
-2.897387141951667
-3.047952817442659
-2.910018923842297
-3.060928367304928
-2.918946018966662
-3.067695682939076
-3.067812652032666
-3.071001537535051
-3.079045842441793
-2.944158833892945
-62.215311367699584
-2.971852557099735
-2.972575174585346
-3.133901957685304
-2.986236440

21886    None
22651    None
21824    None
50880    None
21757    None
         ... 
51442    None
51417    None
53200    None
53088    None
55085    None
Length: 86085, dtype: object

In [48]:
total_cost

8182613.7371026045

In [49]:
total_count

1285